In [1]:
import os
import numpy as np
import pandas as pd

import keras
from keras.layers import *
from keras.optimizers import *
from keras.models import Model

from keras import backend as K

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df1 = pd.read_csv('/content/drive/MyDrive/med_train.csv',header=None)

In [4]:
df2 = pd.read_csv('/content/drive/MyDrive/med_val.csv',header=None)

In [7]:
df3 = pd.concat([df1, df2], ignore_index=True)

In [36]:
df1

,0,1,2,3,4,5,6,7,8,9,...,1304,1305,1306,1307,1308,1309,1310,1311,1312,1313
0,353,4.11,2005,1826,1872,69,4.0,876,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,433,3.92,22,19,33,12,3.0,369,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,154,3.69,2660,644,1003,9,3.5,45,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,668,4.11,1214,180,826,407,4.5,55,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,304,3.60,14,9,8,11,4.0,64,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455849,951,3.47,16440,7770,15239,81,4.0,1830,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
455850,63,3.63,8,3,0,5,1.0,32,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
455851,902,3.71,462,152,345,127,3.0,591,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
455852,52,3.33,27,10,23,1,4.0,453,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
df1.shape

(455854, 1314)

In [33]:
df2.shape

(142044, 1314)

In [8]:
#df_train = df1.iloc[:, 9:] # cate
#df_val = df2.iloc[:, 9:] #cate
df_all = df3.iloc[:, 9:] #cate

In [9]:
#cate_train = np.array(df_train) #dtype = np.int32
#cate_val = np.array(df_val) #dtype = np.int32
cate_all = np.array(df_all)

In [10]:
cate_all.shape

(597898, 1305)

In [11]:
#cate_train = np.nan_to_num(cate_train)
#cate_val = np.nan_to_num(cate_val)
cate_all = np.nan_to_num(cate_all)

In [12]:
#cate_train = np.array(cate_train,dtype = np.int32)
#cate_val = np.array(cate_val,dtype = np.int32)
cate_all = np.array(cate_all,dtype = np.int32)

In [16]:
def NET():
  inp = Input(shape=(cate_all.shape[1],))
  in1 = Dense(16,name = 'med')(inp)
  #in2 = Dense(128,activation = 'relu')(in1) #
  x1 = Dense(cate_all.shape[1],activation='sigmoid')(in1)

  model = Model(inp, x1)
  opt1 = Adam(learning_rate=0.001,clipvalue=1.0)
  model.compile(loss='binary_crossentropy', optimizer=opt1)
  return model

In [17]:
model = NET()
model.fit(cate_all, cate_all, batch_size=5,epochs=2)

Epoch 1/2
119580/119580 [==============================] - 256s 2ms/step - loss: 0.0029
Epoch 2/2
119580/119580 [==============================] - 258s 2ms/step - loss: 7.7375e-04


In [30]:
model.save('model_autoencoder.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
model = NET()
model.load_weights('model_autoencoder.h5')

In [20]:
get_med_output= K.function([model.layers[0].input],[model.layers[1].output])

In [21]:
med_output = get_med_output([cate_all])[0]

In [22]:
med_output.shape

(597898, 16)

In [23]:
med_output

array([[-5.2207603e+00,  5.3115797e+00,  2.0551338e+00, ...,
         3.5873342e+00,  3.3795857e+00,  3.6311381e+00],
       [-5.5129509e+00,  8.4058809e+00, -3.1341066e+00, ...,
         2.4104488e+00, -7.3395389e-01,  3.2443821e+00],
       [ 9.2771387e-01,  3.0279896e+00,  3.1140864e+00, ...,
         5.3527994e+00,  6.7152710e+00, -5.1193790e+00],
       ...,
       [-4.8645110e+00,  3.7604992e+00,  3.5207369e+00, ...,
         5.2731490e+00,  3.1922598e+00, -7.7576435e-01],
       [-9.9938190e-01,  6.7829806e-01,  4.6546888e+00, ...,
         2.8262269e-01,  8.3466768e-03,  1.9125655e+00],
       [ 1.2932770e+00,  2.1597950e+00,  2.2535939e+00, ...,
        -1.3160288e-02,  2.9243479e+00,  1.3529160e+00]], dtype=float32)

In [27]:
df_p1 = df3.iloc[:,:9]

In [25]:
df_p2 = pd.DataFrame(med_output)

In [28]:
df_p = pd.concat([df_p1, df_p2], axis=1)

In [29]:
df_p

,0,1,2,3,4,5,6,7,8,0,...,6,7,8,9,10,11,12,13,14,15
0,353,4.11,2005,1826,1872,69,4.0,876,1,-5.220760,...,-1.679511,1.139333,-0.272802,0.488504,0.106634,-1.335674,2.052226,3.587334,3.379586,3.631138
1,433,3.92,22,19,33,12,3.0,369,1,-5.512951,...,-2.707452,1.854930,-0.416277,0.074615,2.785981,-1.270513,4.779823,2.410449,-0.733954,3.244382
2,154,3.69,2660,644,1003,9,3.5,45,1,0.927714,...,2.011328,-2.596559,-2.321681,-4.196326,1.416534,0.053120,1.911880,5.352799,6.715271,-5.119379
3,668,4.11,1214,180,826,407,4.5,55,1,-1.570636,...,-2.919754,-1.997582,0.601533,0.380995,2.434745,3.207492,-3.739185,-0.152512,-2.987231,1.881980
4,304,3.60,14,9,8,11,4.0,64,1,1.293277,...,0.691412,-2.699798,-2.225234,-3.151780,2.275782,-0.145639,1.263259,-0.013160,2.924348,1.352916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597893,56,3.34,24,6,22,5,3.5,219,1,0.175555,...,-3.202675,-2.662028,-1.374436,5.233773,-1.136648,3.531240,1.699956,0.957610,-1.734966,-2.027642
597894,59,3.47,36,3,0,0,3.0,350,1,1.757083,...,-1.004217,-7.049621,4.036958,1.842802,3.878437,2.528154,5.662129,1.249435,4.135918,1.722216
597895,123,3.51,13,1,1,3,4.0,904,1,-4.864511,...,-4.487249,0.471390,-5.847411,-2.627329,-4.397143,2.861820,3.993481,5.273149,3.192260,-0.775764
597896,55,3.88,4,1,0,1,4.0,104,1,-0.999382,...,-1.763625,-1.004982,-4.004557,-1.430568,-1.053177,2.175006,-2.688200,0.282623,0.008347,1.912565


In [31]:
df_p.to_csv('embed_features.csv', index=False)